In [1]:
# Project Path Setup

import sys
import os
from pathlib import Path

# Define project root (works in notebooks)
PROJECT_ROOT = Path().resolve().parent

# Set working directory to project root
os.chdir(PROJECT_ROOT)

# Add project root to sys.path
sys.path.insert(0, str(PROJECT_ROOT))

print("✅ PROJECT_ROOT:", PROJECT_ROOT)
print("📂 Working directory:", Path.cwd())

✅ PROJECT_ROOT: C:\Users\rajku\Desktop\enterprise_knowledge_assistant
📂 Working directory: C:\Users\rajku\Desktop\enterprise_knowledge_assistant


In [2]:
# import libraries and orchestration

from pathlib import Path
import numpy as np
import config
import pickle

from sentence_transformers import SentenceTransformer
from langchain_core.documents import Document

from src.ingestion.pdf_loader import ingest_pdf
from src.chunking.text_chunker import chunk_documents
from src.embeddings.embedding_utils import load_embedding_model
from src.embeddings.embedding_utils import generate_embeddings

from src.vector_store.faiss_store import (
    build_faiss_index,
    save_vector_store,
    load_vector_store,
)   


In [3]:
# 📌 FAISS works directly on NumPy arrays — no framework magic.

In [3]:
# Ingest ALL PDFs

RAW_DOCS_DIR = Path("data/documents")

documents = []

for pdf_path in RAW_DOCS_DIR.glob("*.pdf"):
    print(f"\n📄 Ingesting: {pdf_path.name}")
    docs = ingest_pdf(
        pdf_path,
        show_preview=False,
        verbose=False
    )
    documents.extend(docs)

print(f"\n✅ Total pages ingested: {len(documents)}")



📄 Ingesting: 2019Q2CorrosioninsightofironandbismuthaddedSn1Ag05Culeadfreesolderalloy2.pdf

📄 Ingesting: A_systematic_survey_of_RUM_process_parameter_optim.pdf

📄 Ingesting: Comparative_Analysis_of_SAC_Solder_Alloys_for_Enha.pdf

📄 Ingesting: Corrosion_Behavior_Analysis_of_Novel_Sn-25Ag-10Bi-.pdf

📄 Ingesting: Effects_of_bismuth_on_the_microstructure_and_crack (1).pdf

📄 Ingesting: Effects_of_bismuth_on_the_microstructure_and_crack.pdf

📄 Ingesting: Effects_of_Bi_and_In_Addition_on_Microstructure_an.pdf

📄 Ingesting: Enhancing_sustainability_in_soldering_effects_of_r.pdf

📄 Ingesting: Laser_science_and_neural_networks-2020 (1).pdf

📄 Ingesting: Laser_science_and_neural_networks-2020.pdf

📄 Ingesting: Neural-fuzzy_machine_learning_approach_for_the_fat.pdf

📄 Ingesting: NovelmachinelearningdrivendesignstrategyforhighstrengthZnAlloysoptimizationwithmultipleconstraints.pdf

📄 Ingesting: s41524-023-01150-0.pdf

📄 Ingesting: s41524-024-01385-5.pdf

📄 Ingesting: s41524-024-01480-7.pdf

📄 Inge

In [4]:
# chunk documents

chunked_documents = chunk_documents(
    documents,
    chunk_size=1000,
    chunk_overlap=200,
    preview_chunks=0,
    verbose=True
)

print(f"\n✅ Total chunks created: {len(chunked_documents)}")


✂️ Chunking started
Chunk size     : 1000
Chunk overlap  : 200
Total docs in  : 432
📦 Total chunks created: 1737

✅ Total chunks created: 1737


In [5]:
# Load Embedding Model

embedding_model = load_embedding_model(config.EMBEDDING_MODEL_NAME)



In [6]:
# Generate Embeddings

embeddings = generate_embeddings(
    documents=chunked_documents,
    model=embedding_model,
    show_progress=True
)



Batches:   0%|          | 0/55 [00:00<?, ?it/s]

In [7]:
# build faiss index

faiss_index = build_faiss_index(embeddings)

✅ FAISS index built with 1737 vectors


In [8]:
# # save vector store using callable function

# VECTOR_STORE_PATH = PROJECT_ROOT / "data" / "processed" / "vector_store.pkl"


save_vector_store(
    index=faiss_index,
    documents=chunked_documents,
    save_path=config.VECTOR_STORE_PATH
)



💾 Vector store saved at:
C:\Users\rajku\Desktop\enterprise_knowledge_assistant\data\processed\vector_store.pkl
